In [1]:
import pandas as pd

results_df = pd.read_csv("../results.csv")
results_df.head()

,plan_found,variation,error,score,complete,num_plans,plan_library_size
0,0,21,True,0,False,0,24
1,1,22,False,100,True,6,24
2,1,23,True,3,False,4,24
3,1,24,False,100,True,8,24
4,1,25,True,3,False,5,24


In [2]:
size = []
score = []
for group, group_df in results_df.groupby(["plan_library_size"]):
    print(f"Plan Library size: {group}")
    print(f"Score: {group_df['score'].mean()}")

Plan Library size: 15
Score: 11.444444444444445
Plan Library size: 18
Score: 33.666666666666664
Plan Library size: 21
Score: 55.888888888888886
Plan Library size: 24
Score: 67.33333333333333


/tmp/ipykernel_22018/3846244346.py:1: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for group, group_df in results_df.groupby(["plan_library_size"]):


In [ ]:
import matplotlib.pyplot as plt

import numpy as np

plt